In [23]:
import os
os.environ['THEANO_FLAGS'] = 'device=cuda'
from machinedesign.autoencoder.interface import load
import numpy as np
import keras.backend as K
from scipy.linalg import sqrtm
import theano.tensor as T
from keras.models import Model
from keras.layers import Input

from machinedesign.viz import grid_of_images_default
from skimage.io import imsave
from machinedesign.autoencoder.interface import _apply_noise, _apply_binarization
def gaussian_kernel(A, B, sigmas=[1], backend=K):
    A_ = A[:, None, :]
    B_ = B[None, :, :]
    out = 0
    for sigma in sigmas:
        m = backend.abs(A_ - B_).sum(axis=2)
        #out = backend.abs(-m / (2 * sigma**2))
        out += m
    return out.mean()


def compute_mmd(X, Y, kernel=gaussian_kernel):
    # https://arxiv.org/pdf/1502.02761.pdf
    phi = kernel
    a = phi(X, X)
    b = phi(X, Y)
    c = phi(Y, Y)
    N = X.shape[0]
    M = Y.shape[0]
    mmd_sqr = (1. / (N**2)) * a - (2. / (N * M)) * b + (1. / M**2) * c
    return mmd_sqr


def compute_frechet(X, Y):
    # https://arxiv.org/pdf/1706.08500.pdf
    X = X.reshape((X.shape[0], -1))
    Y = Y.reshape((Y.shape[0], -1))

    mu_x = X.mean(axis=0)
    mu_y = Y.mean(axis=0)

    cov_x = T.cov(X.T)
    cov_y = T.cov(Y.T)

    return ((mu_x - mu_y)**2).sum() + T.trace(cov_x + cov_y -  2 * sqrtm(np.dot(cov_x, cov_y)))

In [2]:
dataset = '../../data/digits.npz'
data = np.load(dataset)
X = data['X']
y = data['y'] if 'y' in data else None
labels = y
X = X.astype(np.float32)
X /= 255.

In [34]:
#folder = '../results/mnist'
folder = '../results/jobs/c66d4d7f3ef76fd52b2c6b354e94c3d0'

In [35]:
model = load('../results/{}'.format(folder))

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 28, 28)         0         
_________________________________________________________________
model_1 (Model)              (None, 1, 28, 28)         0         
_________________________________________________________________
model_2 (Model)              (None, 1, 28, 28)         1645441   
Total params: 1,645,441
Trainable params: 1,645,441
Non-trainable params: 0
_________________________________________________________________


In [37]:
#mnist
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = Model(inp, hid)

x = Input(model.layers[0].input_shape[1:])
y = Input(model.layers[0].input_shape[1:])
hx = enc(x)
hy = enc(y)
hx = hx.reshape((hx.shape[0], -1))
hy = hy.reshape((hy.shape[0], -1))

#d = compute_mmd(hx, hy)
d = (( (hx).mean(axis=0) - (hy).mean(axis=0))**2).mean()
g = K.gradients(d, x)
get_grad = K.function([x, y], [g])
get_loss = K.function([x, y], [d])

ValueError: No such layer: activation_3

In [99]:
x = np.random.uniform(size=(9, 1, 28, 28))
y = X[labels==0][0:100]
for i in range(100):
    g, = get_grad([x, y])
    x += 0.1 * (model.predict(x) - x) -  2*g
    np.clip(x, 0, 1)
    l, = get_loss([x, y])
    print(l)
    if i % 10 == 0:
        print(l)

3.9829885959625244
3.9829885959625244
3.3729002475738525
2.882402181625366
2.487187385559082
2.161576509475708
1.8963593244552612
1.6709649562835693
1.4900469779968262
1.3492273092269897
1.2361449003219604
1.1525599956512451
1.1525599956512451
1.0816725492477417
1.0179885625839233
0.9578542709350586
0.904114842414856
0.8587921857833862
0.8226959705352783
0.7849229574203491
0.7502723336219788
0.7238856554031372
0.6990135312080383
0.6990135312080383
0.6772550344467163
0.6528810262680054
0.6326917409896851
0.6160693168640137
0.5985620021820068
0.5839024782180786
0.5720347762107849
0.5643101930618286
0.5583369135856628
0.5521355271339417
0.5521355271339417
0.5462051033973694
0.541883111000061
0.5367454290390015
0.5338890552520752
0.5300807952880859
0.5265278220176697
0.5234202146530151
0.5186738967895508
0.5140268802642822
0.5102618336677551
0.5102618336677551
0.5065734386444092
0.5035746097564697
0.4964268207550049
0.48893484473228455
0.48269855976104736
0.4778245687484741
0.4737517833709

In [100]:
xx = x
for _ in range(0):
    xx = model.predict(xx)
im = grid_of_images_default(xx)
imsave('out.png', im)

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [38]:
x = np.random.uniform(size=(100, 1, 28, 28))
for _ in range(100):
    #x = _apply_noise('salt_and_pepper', {'proba': 0.1}, x)
    x = model.predict(x)
    #x = _apply_binarization('sample_bernoulli', {}, x)
x = model.predict(x)
im = grid_of_images_default(x)
imsave('out.png', im)

/home/mcherti/miniconda/envs/py3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
